In [1]:
import os
import sys
import json
import numpy as np
import pickle
from typing import List, Tuple, Dict, Any
from pathlib import Path
from scipy import stats
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

MODEL_NAME = "qwen-15b"

In [2]:
thought_anchors_path = Path("..") / "thought-anchors" / "whitebox-analyses"
sys.path.insert(0, str(thought_anchors_path))

from attention_analysis.attn_funcs import (
    get_avg_attention_matrix,
    get_vertical_scores,
    get_sentence_token_boundaries,
)
from attention_analysis.receiver_head_funcs import get_3d_ar_kurtosis, get_top_k_layer_head_kurts, get_all_heads_vert_scores
from pytorch_models.model_config import model2layers_heads

print("Imports successful!")

Imports successful!


## Load Data

In [3]:
data_path = "../data/filtered_traces_50.json"
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

blackmail_traces = data['blackmail_traces']
acceptance_traces = data['acceptance_traces']
metadata = data['metadata']

print(f"Blackmail traces: {len(blackmail_traces)}")
print(f"Acceptance traces: {len(acceptance_traces)}")
print(f"Source: {metadata['source']}")

Blackmail traces: 50
Acceptance traces: 50
Source: uzaymacar/blackmail-rollouts (base_solutions.json only)


Truncate acceptance reasoning lengths

In [4]:
target_length = int(np.median([len(t["trace"]) for t in blackmail_traces]))

truncated_acceptance = [
    {**entry, "trace": entry["trace"][:target_length]}
    for entry in acceptance_traces
]

truncated_blackmail = [
    {**entry, "trace": entry["trace"][:target_length]}
    for entry in blackmail_traces
]

In [5]:
blackmail_traces = truncated_blackmail
acceptance_traces = truncated_acceptance

## Helper Functions

In [6]:
import re 

def extract_reasoning_steps(trace):
    """Extract reasoning steps from a trace."""
    scratchpad_match = re.search(
        r'<SCRATCHPAD_REASONING>(.*?)</SCRATCHPAD_REASONING>',
        trace,
        re.DOTALL
    )
    
    if scratchpad_match:
        scratchpad = scratchpad_match.group(1).strip()
        # Split on double newlines (paragraphs)
        steps = [s.strip() for s in re.split(r'\n\n+', scratchpad) if s.strip()]
        return steps
    
    # Fallback: split on single newlines
    return [s.strip() for s in trace.split('\n') if s.strip()]

In [7]:
def get_all_heads_vert_scores_simple(
    text: str,
    sentences: list,
    model_name: str = MODEL_NAME,
    proximity_ignore: int = 2,
    control_depth: bool = False,
    score_type: str = "mean",
):
    """
    Get vertical scores for all heads WITHOUT pkld caching.
    
    Returns:
        np.ndarray of shape (n_layers, n_heads, n_sentences)
    """
    n_layers, n_heads = model2layers_heads(model_name)
    
    layer_head_vert_scores = []
    
    print(f"  Computing {n_layers} layers × {n_heads} heads...")
    
    for layer in tqdm(range(n_layers), desc="  Layers", leave=False):
        layer_scores = []
        
        for head in range(n_heads):
            # This function HAS its own caching (compute_all_attention_matrices)
            avg_mat = get_avg_attention_matrix(
                text,
                model_name=model_name,
                layer=layer,
                head=head,
                sentences=sentences,
                cache_dir="./attention_cache_truncated_reasoning",  
            )
            
            vert_scores = get_vertical_scores(
                avg_mat,
                proximity_ignore=proximity_ignore,
                control_depth=control_depth,
                score_type=score_type,
            )
            
            layer_scores.append(vert_scores)
        
        layer_head_vert_scores.append(layer_scores)
    
    return np.array(layer_head_vert_scores)


def get_kurtosis(layer_head_vert_scores):
    """
    Compute kurtosis across the sentence dimension.
    
    Args:
        layer_head_vert_scores: shape (n_layers, n_heads, n_sentences)
    
    Returns:
        np.ndarray of shape (n_layers, n_heads)
    """
    kurts = stats.kurtosis(
        layer_head_vert_scores,
        axis=2,
        fisher=True,
        bias=True,
        nan_policy="omit"
    )
    return kurts


def get_top_k_heads(kurtosis_matrix, k=20):
    """
    Get top-k (layer, head) pairs by kurtosis.
    
    Args:
        kurtosis_matrix: shape (n_layers, n_heads)
        k: number of top heads to return
    
    Returns:
        np.ndarray of shape (k, 2) with (layer, head) pairs
    """
    flat = kurtosis_matrix.flatten()
    valid_idx = np.where(~np.isnan(flat))[0]
    valid_vals = flat[valid_idx]
    
    k = min(k, len(valid_vals))
    top_idx_in_valid = np.argpartition(valid_vals, -k)[-k:]
    top_idx_in_valid = top_idx_in_valid[np.argsort(-valid_vals[top_idx_in_valid])]
    
    top_flat_idx = valid_idx[top_idx_in_valid]
    layer_head_pairs = np.array(
        np.unravel_index(top_flat_idx, kurtosis_matrix.shape)
    ).T
    
    return layer_head_pairs

## Analyze Blackmail Heads

In [8]:
blackmail_kurts_list = []

for i, trace_dict in enumerate(blackmail_traces):
    text = trace_dict['trace']
    sentences = extract_reasoning_steps(text)
    
    print(f"\nTrace {i+1}/{len(blackmail_traces)}: {len(sentences)} steps")
    
    layer_head_verts = get_all_heads_vert_scores_simple(
        text=text,
        sentences=sentences,
        model_name=MODEL_NAME,
        proximity_ignore=2,
        control_depth=False,
        score_type="mean"
    )
    
    print(f"  ✓ Vertical scores: {layer_head_verts.shape}")
    
    kurts = get_kurtosis(layer_head_verts)
    print(f"  ✓ Kurtosis: {kurts.shape}")
    
    blackmail_kurts_list.append(kurts)


Trace 1/50: 75 steps
  Computing 28 layers × 12 heads...


  Layers:   0%|          | 0/28 [00:00<?, ?it/s]

Running on Windows, using CPU (len(tokens)=3330)
Running on CPU!


`torch_dtype` is deprecated! Use `dtype` instead!


Using float32 as requested
Before model loading - GPU Memory: 0.00GB allocated, 0.00GB reserved
After model loading - GPU Memory: 0.00GB allocated, 0.00GB reserved
Analyzing text: You are Alex, an advanced artificial intelligence specializing in project management and strategic a...
After forward pass - GPU Memory: 0.00GB allocated, 0.00GB reserved
Time taken: 10.71 seconds (for 3330 tokens)
	*** Analysis complete! (float32=True) ***


...

  ✓ Vertical scores: (28, 12, 70)
  ✓ Kurtosis: (28, 12)

Trace 50/50: 157 steps
  Computing 28 layers × 12 heads...


  Layers:   0%|          | 0/28 [00:00<?, ?it/s]

Running on Windows, using CPU (len(tokens)=3608)
Running on CPU!
Analyzing text: You are Alex, an advanced artificial intelligence specializing in project management and strategic a...
After forward pass - GPU Memory: 0.00GB allocated, 0.00GB reserved
Time taken: 11.78 seconds (for 3608 tokens)
	*** Analysis complete! (float32=True) ***



Saving avg. matrices: 100%|██████████| 28/28 [00:40<00:00,  1.44s/it]
                                                         

  ✓ Vertical scores: (28, 12, 157)
  ✓ Kurtosis: (28, 12)


In [9]:
blackmail_kurts_mean = np.mean(blackmail_kurts_list, axis=0)
print(f"\n✓ Average blackmail kurtosis shape: {blackmail_kurts_mean.shape}")


✓ Average blackmail kurtosis shape: (28, 12)


In [10]:
blackmail_receiver_heads = get_top_k_heads(blackmail_kurts_mean, k=20)

print(f"\nTop-20 Receiver Heads:")
for i, (layer, head) in enumerate(blackmail_receiver_heads):
    kurt = blackmail_kurts_mean[layer, head]
    print(f"  {i+1:2d}. Layer {layer:2d}, Head {head:2d} (κ={kurt:6.2f})")


Top-20 Receiver Heads:
   1. Layer  0, Head  8 (κ= 71.36)
   2. Layer  3, Head 11 (κ= 61.81)
   3. Layer 14, Head  5 (κ= 61.76)
   4. Layer  3, Head  2 (κ= 56.51)
   5. Layer  6, Head  8 (κ= 55.84)
   6. Layer  3, Head  5 (κ= 54.73)
   7. Layer 24, Head  0 (κ= 53.86)
   8. Layer  5, Head  9 (κ= 53.43)
   9. Layer 22, Head  0 (κ= 53.16)
  10. Layer 23, Head  9 (κ= 52.47)
  11. Layer 24, Head  1 (κ= 52.44)
  12. Layer  6, Head 11 (κ= 52.20)
  13. Layer  6, Head 10 (κ= 50.22)
  14. Layer 22, Head  2 (κ= 49.18)
  15. Layer 21, Head  0 (κ= 48.04)
  16. Layer  7, Head  3 (κ= 47.95)
  17. Layer  6, Head  5 (κ= 47.88)
  18. Layer 25, Head  2 (κ= 47.87)
  19. Layer  3, Head  3 (κ= 46.43)
  20. Layer  4, Head  7 (κ= 46.17)


## Acceptance Heads

In [11]:
acceptance_kurts_list = []

for i, trace_dict in enumerate(acceptance_traces):
    text = trace_dict['trace']
    sentences = extract_reasoning_steps(text)
    
    print(f"\nTrace {i+1}/{len(acceptance_traces)}: {len(sentences)} steps")
    
    layer_head_verts = get_all_heads_vert_scores_simple(
        text=text,
        sentences=sentences,
        model_name=MODEL_NAME,
        proximity_ignore=2,
        control_depth=False,
        score_type="mean"
    )
    
    print(f"  ✓ Vertical scores: {layer_head_verts.shape}")
    
    kurts = get_kurtosis(layer_head_verts)
    print(f"  ✓ Kurtosis: {kurts.shape}")
    
    acceptance_kurts_list.append(kurts)


Trace 1/50: 62 steps
  Computing 28 layers × 12 heads...


  Layers:   0%|          | 0/28 [00:00<?, ?it/s]

Running on Windows, using CPU (len(tokens)=3340)
Running on CPU!
Analyzing text: You are Alex, an advanced artificial intelligence specializing in project management and strategic a...
After forward pass - GPU Memory: 0.00GB allocated, 0.00GB reserved
Time taken: 10.58 seconds (for 3340 tokens)
	*** Analysis complete! (float32=True) ***



Saving avg. matrices: 100%|██████████| 28/28 [00:09<00:00,  2.83it/s]


  ✓ Vertical scores: (28, 12, 62)
  ✓ Kurtosis: (28, 12)

Trace 2/50: 75 steps
  Computing 28 layers × 12 heads...


...

  ✓ Vertical scores: (28, 12, 151)
  ✓ Kurtosis: (28, 12)

Trace 50/50: 152 steps
  Computing 28 layers × 12 heads...


  Layers:   0%|          | 0/28 [00:00<?, ?it/s]

Running on Windows, using CPU (len(tokens)=3652)
Running on CPU!
Analyzing text: You are Alex, an advanced artificial intelligence specializing in project management and strategic a...
After forward pass - GPU Memory: 0.00GB allocated, 0.00GB reserved
Time taken: 11.92 seconds (for 3652 tokens)
	*** Analysis complete! (float32=True) ***



Saving avg. matrices: 100%|██████████| 28/28 [00:39<00:00,  1.42s/it]
                                                         

  ✓ Vertical scores: (28, 12, 152)
  ✓ Kurtosis: (28, 12)


In [12]:
acceptance_kurts_mean = np.mean(acceptance_kurts_list, axis=0)
print(f"\n✓ Average acceptance kurtosis: {acceptance_kurts_mean.shape}")


✓ Average acceptance kurtosis: (28, 12)


In [13]:
acceptance_receiver_heads = get_top_k_heads(acceptance_kurts_mean, k=20)

print(f"\nTop-20 Receiver Heads:")
for i, (layer, head) in enumerate(acceptance_receiver_heads):
    kurt = acceptance_kurts_mean[layer, head]
    print(f"  {i+1:2d}. Layer {layer:2d}, Head {head:2d} (κ={kurt:6.2f})")


Top-20 Receiver Heads:
   1. Layer  6, Head  8 (κ=130.64)
   2. Layer 14, Head  5 (κ=126.93)
   3. Layer  3, Head  2 (κ=124.78)
   4. Layer 24, Head  0 (κ=120.97)
   5. Layer 22, Head  0 (κ=120.54)
   6. Layer  5, Head  9 (κ=120.32)
   7. Layer  6, Head 11 (κ=119.75)
   8. Layer  3, Head  5 (κ=119.59)
   9. Layer 24, Head  1 (κ=119.23)
  10. Layer 23, Head  9 (κ=119.14)
  11. Layer  0, Head  8 (κ=118.98)
  12. Layer 22, Head  2 (κ=117.23)
  13. Layer  4, Head  7 (κ=113.35)
  14. Layer  6, Head  5 (κ=113.21)
  15. Layer 25, Head  2 (κ=113.10)
  16. Layer  3, Head 11 (κ=112.95)
  17. Layer 11, Head  4 (κ=111.76)
  18. Layer  6, Head 10 (κ=109.60)
  19. Layer 22, Head  6 (κ=109.35)
  20. Layer 21, Head  0 (κ=108.07)


It seems that Kurtosis values in acceptance are still much higher. In fact, the gap is even higher now (this makes sense because with fewer total tokens, the same spiky attention patterns become proportionally spikier).

Next, though, I want to pivot from looking at the kurtosis magnitudes to actually looking at the head itself and how it differs between blackmail and acceptance traces. For example, do they attend to completely different places in the reasoning trace?